<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🛒Shop</font>](https://www.kaggle.com/competitions/19feb24jh-shop/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 19feb24jh-shop    # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 19feb24jh-shop
Using competition: 19feb24jh-shop
  teamId  teamName      submissionDate       score         
--------  ------------  -------------------  ------------  
11689393  6 Shop        2024-03-03 03:35:45  0.9876800000  
11693486  7 Shop        2024-03-04 02:21:46  0.9836400000  
11690103  1 Stop Shop🛒  2024-03-03 21:44:03  0.9801600000  
11689194  8 Shop        2024-03-03 05:06:42  0.9754800000  
11701157  5 Shop        2024-03-04 02:14:38  0.9683200000  
11693808  2 Shop        2024-03-03 05:02:50  0.9536400000  
11702354  3 Shop        2024-03-03 02:56:14  0.9272000000  
11701042  4 Shop        2024-03-04 02:16:18  0.8809200000  
11529895  🛒Baseline🐍    2023-09-09 16:47:47  0.8044400000  


In [ ]:
!pip -q install -U tensorflow_addons > log

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 582 ms, sys: 24.9 ms, total: 607 ms
Wall time: 808 ms


In [ ]:
df = pd.read_csv('XY_Shop.csv'); df

,Adm,AdmDur,Inf,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
0,0,0.00,0,0.0,18,132.99,3.82e-02,5.45e-02,0.0,0.0,4,3,1,1,2,0,1,NaN
1,1,0.00,0,0.0,37,1150.20,1.25e-03,3.03e-02,0.0,0.0,11,2,2,4,2,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499998,0,0.00,0,0.0,27,1185.14,0.00e+00,1.59e-03,0.0,0.0,5,2,2,2,3,0,1,0.0
499999,6,51.36,0,0.0,59,1898.21,0.00e+00,3.22e-03,0.0,0.0,12,2,2,2,1,0,0,0.0


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Adm      500000 non-null  int64  
 1   AdmDur   500000 non-null  float64
 2   Inf      500000 non-null  int64  
 3   InfDur   500000 non-null  float64
 4   Prd      500000 non-null  int64  
 5   PrdDur   500000 non-null  float64
 6   BncRt    500000 non-null  float64
 7   ExtRt    500000 non-null  float64
 8   PgVal    500000 non-null  float64
 9   SpclDay  500000 non-null  float64
 10  Mo       500000 non-null  int64  
 11  OS       500000 non-null  int64  
 12  Bsr      500000 non-null  int64  
 13  Rgn      500000 non-null  int64  
 14  TfcTp    500000 non-null  int64  
 15  VstTp    500000 non-null  int64  
 16  Wkd      500000 non-null  int64  
 17  Rev      450000 non-null  float64
dtypes: float64(8), int64(10)
memory usage: 68.7 MB


In [ ]:
vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
tXY = df.query('Rev==Rev')                     # slice training sample
tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
NumFeatures = list(tX.select_dtypes(include='float').columns)
print('Numeric features: ', NumFeatures)       # numeric/quantitative feature names

Numeric features:  ['AdmDur', 'InfDur', 'PrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay']


In [ ]:
# def ScatterCorrHist(df):
#   def corrdot(*args, **kwargs):
#     # credit: https://stackoverflow.com/questions/48139899
#     corr_r = args[0].corr(args[1], 'pearson')
#     corr_text = f"{corr_r:2.2f}".replace("0.", ".")
#     ax = plt.gca();
#     ax.set_axis_off();
#     msz = abs(corr_r) * 5000   # marker size
#     fsz = abs(corr_r) * 40 + 5 # font size
#     ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
#     ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

#   sns.set(style='white', font_scale=.8);
#   g = sns.PairGrid(df, aspect=1, diag_sharey=False);
#   g.fig.set_size_inches(20,10)
#   g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
#   g.map_diag(sns.histplot, kde_kws={'color':'black'});
#   g.map_upper(corrdot);
#   g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
#   _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
#   _ = plt.show();

# df_ = tXY.loc[(tXY[NumFeatures]>0).all(axis=1), NumFeatures+['Rev']].sample(n=100, random_state=0)
# # df_ = df.select_dtypes(include='float').query('AdmDur>0 and InfDur>0 and PrdDur>0 and BncRt>0 and ExtRt>0 and PgVal>0 and SpclDay>0').sample(n=100, random_state=0)
# ScatterCorrHist(df_)

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Build a model</b></h3></font>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA
from keras import backend as K

# Data preprocessing
# Determine the minority and majority classes
minority_class = tY.value_counts().idxmin()
majority_class = tY.value_counts().idxmax()

# Initialize the RandomOverSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=42)

# Oversampling the minority class in the training set
tX_resampled, tY_resampled = ros.fit_resample(tX, tY)


# Numeric features should have more - to be scaled
NumFeatures = ['Adm', 'Inf', 'Prd'] + NumFeatures
# Categorical features to be one-hot encoded
CatFeatures = ['Mo', 'OS', 'Bsr', 'Rgn', 'TfcTp', 'VstTp', 'Wkd']
#CatFeatures = ['Mo', 'Rgn', 'TfcTp', 'VstTp', 'Wkd']

# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False), CatFeatures),
        ('num', StandardScaler(), NumFeatures)
    ])

# Add PCA to the pipeline
# Set n_components to preserve 85% of the information
pca = PCA(n_components=0.85)

# Construct the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
    #('pca', pca)
])

# Fit the pipeline on the data
pipeline.fit(tX_resampled)
# Transform the data
X_transformed = pipeline.transform(tX_resampled)
vX_transformed = pipeline.transform(vX)

# Check the shape of the transformed data
print("Transformed shape:", X_transformed.shape)
print("Transformed shape:", vX_transformed.shape)
print(type(X_transformed))



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Mo', 'OS', 'Bsr', 'Rgn',
                                                   'TfcTp', 'VstTp', 'Wkd']),
                                                 ('num', StandardScaler(),
                                                  ['Adm', 'Inf', 'Prd',
                                                   'AdmDur', 'InfDur', 'PrdDur',
                                                   'BncRt', 'ExtRt', 'PgVal',
                                                   'SpclDay'])]))])

Transformed shape: (698280, 59)
Transformed shape: (50000, 59)
<class 'numpy.ndarray'>


In [ ]:
%%time
tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)

tX_in = X_transformed[:]
tY_in = tY_resampled[:]

m = keras.models.Sequential([
    Flatten(input_shape=[tX_in.shape[1]], name='input'),
    Dense(100, activation="relu", kernel_initializer=Init, name='hidden1'),
    Dense(100, activation="relu", kernel_initializer=Init, name='hidden2'),
    Dense(80, activation="relu", kernel_initializer=Init, name='hidden3'),
    Dense(80, activation="relu", kernel_initializer=Init, name='hidden4'),
    Dense(1, activation='sigmoid', kernel_initializer=Init, name='output')])
m.summary()
m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
hist = m.fit(tX_in, tY_in, epochs=14, validation_split=0.3, batch_size=1000)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 59)                0         
                                                                 
 hidden1 (Dense)             (None, 100)               6000      
                                                                 
 hidden2 (Dense)             (None, 100)               10100     
                                                                 
 hidden3 (Dense)             (None, 80)                8080      
                                                                 
 hidden4 (Dense)             (None, 80)                6480      
                                                                 
 output (Dense)              (None, 1)                 81        
                                                                 
Total params: 30741 (120.08 KB)
Trainable params: 3074

In [ ]:
pY = pd.DataFrame(m.predict(vX_transformed), index=range(1, len(vX_transformed) + 1), columns=['Rev'])
ToCSV(pY.round(0).astype(int), '🛒Baseline🐍')
tXY.Rev.value_counts()/len(tXY)                # distribution of training target level
pY.round(0).astype(int).value_counts()/len(pY) # distribution of test target level

1563/1563 [==============================] - 3s 2ms/step


0.0    0.78
1.0    0.22
Name: Rev, dtype: float64

Rev
0      0.8
1      0.2
dtype: float64

<font color=green><h3><b>$\beta$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Our main item in our preprocessing pipeline was doing standard scaling and one hot encoding on all of our categorical variables. We experimented out all of the other types of scaling, robust, minmax, Moreover, we delved into sampling techniques to address concerns regarding class imbalances, which could lead to biased model predictions.
2. We explored both oversampling and undersampling methods to balance the distribution of classes in our dataset. Eventually, we discovered our model performed best on the oversampling set. We still left out 30% for the validation set because we were worried about overfitting. However, we felt confident when we realized the public leaderboard results matched will with our validation set in terms of accuracy.
3.We also tried using same features ('PgVal', 'ExtRt', ‘InfDur’, 'Mo','Wkd', ‘Rgn'] described in Sakar et al. They used a technique called Minimum Redundancy Maximum Relevance (mRMR). We also tried to implement their version of model called resilient backpropagation algorithm with weight backtracking but couldn’t do it.

</font>

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. After an extensive hyperparameter tuning process, we arrived at our final model configuration. Our exploration involved meticulous experimentation across a spectrum of hyperparameters, including learning rates, activation functions, loss functions, epochs, optimization algorithms, batch sizes, and the number of hidden layers.
2. For our final submission, our model was sequential with 4 hidden layers, using as many epochs as we could but stil stay well within the 1 minute time limit set for the projects. We tried to use GPU backend, which improves the computation time dramatically, but that does not seem to be a guarenteed resource from colab, so please run this program while GPU is available.
3. We also noticed a certain group of items tended to be more often misclassified and strongly classified. To address this, we tried adding weights to the classes, but that negatively impacted accuracy, so we removed it.
</font>

<font color=green><h3><b>$\gamma$. References</b></h3></font>

1. Géron, A. (2019). Hands-on machine learning with Scikit-learn, Keras, and Tensorflow: Concepts, tools, and techniques to build intelligent systems (2nd edition). O'Reilly Media.
2. Oleg Melnikov. (2024). 19feb24jh🛒-Shop. Kaggle. Rules. https://www.kaggle.com/competitions/19feb24jh-shop/rules
3. Sakar, C. O., Polat, S. O., Katircioglu, M., & Kastro, Y. (2019). Real-time prediction of online shoppers’ purchasing intention using multilayer perceptron and LSTM recurrent neural networks. Neural Computing and Applications, 31, 6893-6908.
4. https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html

Packages:
1. Imbalanced-learn. (n.d.-a). RandomOverSampler. https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.RandomOverSampler.html
2. Keras-Tuner. (n.d.-b). kt. https://keras.io/keras_tuner/
3. Scikit-learn. (n.d.-c). ColumnTransformer. https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
4. Scikit-learn. (n.d.-d). MinMaxScaler. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
5. Scikit-learn. (n.d.-e). OneHotEncoder. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
6. Scikit-learn. (n.d.-f). PCA. https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
7. Scikit-learn. (n.d.-g). Pipeline. https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
8. Scikit-learn. (n.d.-h). PowerTransformer. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html
9. Scikit-learn. (n.d.-i). QuantileTransformer. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html
10. Scikit-learn. (n.d.-j). RobustScaler. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
11. Scikit-learn. (n.d.-k). StandardScaler. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 61 sec


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)